In [1]:
import pandas as panda
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
import seaborn
from textstat.textstat import *
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer as VS
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

ModuleNotFoundError: No module named 'textstat'

In [ ]:
dataset = panda.read_excel(r'C:/Users/Acer/Desktop/Text violation detection/Dataset/cartoonConvoTraining.xlsx',index_col=0)
dataset

In [ ]:
#convert float data type to string. Phrase is in object type. for clean it should convert to string
dataset.Phrase=dataset.Phrase.astype(str)

In [ ]:

# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', str(text))
    text = re.sub('[%s]' % re.escape(string.punctuation), '', str(text))

    text = re.sub('\w*\d\w*', '', str(text))
    return text

round1 = lambda x: clean_text_round1(x)

In [34]:

# Let's take a look at the updated text. apply clean_text_round1 method for Phrase field.
data_clean = panda.DataFrame(dataset.Phrase.apply(round1))
data_clean

,Phrase
PhraseNo,
1,yes you baboon baboon baboon baboon
2,did you hear that marge she called me a baboon...
3,nuts
4,mmm hors doovers
5,you stink you and your whole lousy operation s...
...,...
1996,la la l oop
1997,oh well lets sell it anyway
1998,yo check this out black guys drive a car like ...


In [36]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', str(text))
    text = re.sub('\n', '', str(text))
    return text

round2 = lambda x: clean_text_round2(x)

In [37]:
# Let's take a look at the updated text
data_clean = panda.DataFrame(data_clean.Phrase.apply(round2))
data_clean

,Phrase
PhraseNo,
1,yes you baboon baboon baboon baboon
2,did you hear that marge she called me a baboon...
3,nuts
4,mmm hors doovers
5,you stink you and your whole lousy operation s...
...,...
1996,la la l oop
1997,oh well lets sell it anyway
1998,yo check this out black guys drive a car like ...


In [38]:
Phrase = data_clean.Phrase.astype(str)
Phrase

PhraseNo
1                     yes you baboon baboon baboon baboon
2       did you hear that marge she called me a baboon...
3                                                    nuts
4                                        mmm hors doovers
5       you stink you and your whole lousy operation s...
                              ...                        
1996                                          la la l oop
1997                          oh well lets sell it anyway
1998    yo check this out black guys drive a car like ...
1999                       it is time to settle the score
2000                                              shut up
Name: Phrase, Length: 2000, dtype: object

In [40]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

tokens_by_word = ''

tokens_by_word = word_tokenize(data_clean)
tokens_by_word = [w.replace('\n', ' ').replace('- ', '') for w in tokens_by_word]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


TypeError: expected string or bytes-like object

In [ ]:
Final_words = []

In [ ]:

j = 0
while j < len(data_clean):
    filtered_title = normalization(no_user_alpha(clean_text_round2(data_clean['Phrase'][j])))
#     word_Final = ' '.join(filtered_title)
#     Final_words.append(word_Final)
    Final_words.append(filtered_title)
    j += 1
    
Final_words  

# Document-Term Matrix

In [ ]:
# text must be tokenized, meaning broken down into smaller pieces.
# The most common tokenization technique is to break down text into words.
# We can do this using scikit-learn's CountVectorizer
# In addition, with CountVectorizer, we can remove stop words. 
# Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [ ]:

# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.Phrase)
data_dtm = panda.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

In [ ]:
# Let's pickle it for later use
data_dtm.to_pickle("dtm.pkl")